In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 

In [17]:
dfDeliveries = pd.read_csv("../data/IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv("../data/IPL_Matches_2008_2022.csv")

# Data Cleaning


removing matches where D/L is applied


In [21]:
dfMatches['method'].value_counts()

Series([], Name: count, dtype: int64)

In [22]:
dfMatches['method'].unique()

array([nan], dtype=object)

In [20]:
dfMatches = dfMatches[dfMatches['method'].isna()]

cleaning season column


In [23]:
dfMatches['Season'] = dfMatches['Season'].replace({'2020/21': '2020', '2009/10': '2010', '2007/08': '2008'})

cleaning team names


In [24]:
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['WinningTeam']=dfMatches['WinningTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [25]:
dfMatches['Team1']=dfMatches['Team1'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team1']=dfMatches['Team1'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team1']=dfMatches['Team1'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [26]:
dfMatches['Team2']=dfMatches['Team2'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['Team2']=dfMatches['Team2'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['Team2']=dfMatches['Team2'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

In [27]:
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Delhi Daredevils','Delhi Capitals')
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Kings XI Punjab','Punjab Kings') 
dfMatches['TossWinner']=dfMatches['TossWinner'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

# merged data frame


In [29]:
dfMerged = dfDeliveries.merge(dfMatches, on='ID')

In [30]:
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Delhi Daredevils','Delhi Capitals')
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Kings XI Punjab','Punjab Kings') 
dfMerged['BattingTeam']=dfMerged['BattingTeam'].replace('Rising Pune Supergiants','Rising Pune Supergiant')

Bowling Team Column


In [31]:
dfMerged['BowlingTeam'] = np.where(dfMerged['BattingTeam']==dfMerged['Team1'],
                                       dfMerged['Team2'],
                                       dfMerged['Team1'])

In [32]:
dfBatsmen = dfMerged.groupby('batter')\
            .agg({'ID':'nunique','batsman_run':'sum','isWicketDelivery':'sum'})\
            .reset_index()
dfBatsmen.columns = ['Batter','Innings','Runs','Dismissals']
dfBallsFaced = dfMerged[(dfMerged['extra_type']!='wides')].groupby('batter')['ID'].count().reset_index()
dfBallsFaced.columns = ['Batter','Balls Faced']
dfBatter0s = dfMerged[(dfMerged['batsman_run']==0)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter0s.columns = ['Batter','0s']
dfBatter1s = dfMerged[(dfMerged['batsman_run']==1)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter1s.columns = ['Batter','1s']
dfBatter2s = dfMerged[(dfMerged['batsman_run']==2)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter2s.columns = ['Batter','2s']
dfBatter4s = dfMerged[(dfMerged['batsman_run']==4)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter4s.columns = ['Batter','4s']
dfBatter6s = dfMerged[(dfMerged['batsman_run']==6)&(dfMerged['non_boundary']==0)]\
            .groupby('batter')['ID']\
            .count()\
            .reset_index()\
            .sort_values('ID')
dfBatter6s.columns = ['Batter','6s']
dfMatchWise = dfMerged.groupby(['batter','ID'])['batsman_run'].sum().reset_index()
dfHighScore = dfMatchWise.groupby(['batter'])['batsman_run'].max().reset_index()
dfHighScore.columns = ['Batter','High Score']

df25s = dfMatchWise[(dfMatchWise['batsman_run']>=25)&(dfMatchWise['batsman_run']<50)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df25s.columns = ['Batter','25']

df50s = dfMatchWise[(dfMatchWise['batsman_run']>=50)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df50s.columns = ['Batter','50']

df75s = dfMatchWise[(dfMatchWise['batsman_run']>=75)&(dfMatchWise['batsman_run']<100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df75s.columns = ['Batter','75']

df100s = dfMatchWise[(dfMatchWise['batsman_run']>=100)]\
        .groupby('batter')['batsman_run']\
        .count()\
        .reset_index()\
        .sort_values('batsman_run',ascending=False)
df100s.columns = ['Batter','100']

dfBatsmen = dfBatsmen.merge(dfBallsFaced, on='Batter',how='outer').merge(dfBatter0s, on='Batter',how='outer').merge(dfBatter1s, on='Batter',how='outer').merge(dfBatter2s, on='Batter',how='outer').merge(dfBatter4s, on='Batter',how='outer')\
                     .merge(dfBatter6s, on='Batter',how='outer').merge(dfHighScore, on='Batter',how='outer')\
                     .merge(df25s, on='Batter',how='outer').merge(df50s, on='Batter',how='outer').merge(df75s, on='Batter',how='outer').merge(df100s, on='Batter',how='outer')
dfBatsmen['Strike Rate'] = (dfBatsmen['Runs']/dfBatsmen['Balls Faced'])*100
dfBatsmen['Batting Average'] = dfBatsmen['Runs']/dfBatsmen['Dismissals']
dfBatsmen.fillna(0, inplace=True)
dfBatsmen['notout'] = dfBatsmen['Innings'] - dfBatsmen['Dismissals']
dfBatsmen.sort_values(by='notout', ascending=False).head()

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
339,MS Dhoni,201,4895,140,3616,1340.0,1490.0,328.0,337.0,226.0,84,65.0,23.0,3.0,0.0,135.370575,34.964286,61
423,RA Jadeja,156,2486,100,1954,732.0,855.0,180.0,182.0,88.0,62,32.0,2.0,0.0,0.0,127.226203,24.860000,56
27,AB de Villiers,167,5168,123,3395,1105.0,1415.0,268.0,412.0,253.0,133,37.0,41.0,11.0,3.0,152.223859,42.016260,44
136,DJ Bravo,110,1560,69,1204,498.0,447.0,111.0,120.0,66.0,70,13.0,5.0,0.0,0.0,129.568106,22.608696,41
597,YK Pathan,149,3143,108,2195,848.0,864.0,153.0,257.0,154.0,100,31.0,14.0,0.0,1.0,143.189066,29.101852,41


In [33]:
dfBatsmen.head(10)

,Batter,Innings,Runs,Dismissals,Balls Faced,0s,1s,2s,4s,6s,High Score,25,50,75,100,Strike Rate,Batting Average,notout
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24


In [36]:
batsman_stats_df = dfBatsmen.rename(columns={'Batter': 'batter', 'Innings': 'matches_played','Runs':'runs_scored','Dismissals':'dismissals','Balls Faced':'balls_faced',
                                             '0s':'0s_scored','1s':'1s_scored','2s':'2s_scored','4s':'4s_scored','6s':'6s_scored','High Score':'high_score','25':'25_scored',
                                             '50':'50_scored','75':'75_scored','100':'100_scored','Strike Rate':'strike_rate','Batting Average':'batting_average'
                                             })

In [37]:
batsman_stats_df.head(15)

,batter,matches_played,runs_scored,dismissals,balls_faced,0s_scored,1s_scored,2s_scored,4s_scored,6s_scored,high_score,25_scored,50_scored,75_scored,100_scored,strike_rate,batting_average,notout
0,A Ashish Reddy,23,280,15,193,61.0,83.0,20.0,16.0,15.0,36,2.0,0.0,0.0,0.0,145.077720,18.666667,8
1,A Badoni,11,161,9,130,57.0,53.0,11.0,11.0,7.0,54,0.0,1.0,0.0,0.0,123.846154,17.888889,2
2,A Chandila,2,4,1,7,3.0,4.0,0.0,0.0,0.0,4,0.0,0.0,0.0,0.0,57.142857,4.000000,1
3,A Chopra,6,53,5,71,45.0,21.0,2.0,7.0,0.0,24,0.0,0.0,0.0,0.0,74.647887,10.600000,1
4,A Choudhary,3,25,2,20,4.0,13.0,1.0,1.0,1.0,15,0.0,0.0,0.0,0.0,125.000000,12.500000,1
5,A Dananjaya,1,4,0,5,2.0,2.0,1.0,0.0,0.0,4,0.0,0.0,0.0,0.0,80.000000,inf,1
6,A Flintoff,3,62,2,53,24.0,23.0,2.0,5.0,2.0,24,0.0,0.0,0.0,0.0,116.981132,31.000000,1
7,A Kumble,15,35,2,47,24.0,21.0,1.0,3.0,0.0,8,0.0,0.0,0.0,0.0,74.468085,17.500000,13
8,A Manohar,7,108,6,75,30.0,30.0,2.0,14.0,3.0,43,2.0,0.0,0.0,0.0,144.000000,18.000000,1
9,A Mishra,54,362,30,389,182.0,172.0,22.0,29.0,5.0,31,3.0,0.0,0.0,0.0,93.059126,12.066667,24


In [39]:
#explosivity rating = Total no. of boundaries hit / Total no. of balls faced
batsman_stats_df['explosivity_rating'] = round((batsman_stats_df['4s_scored']+batsman_stats_df['6s_scored'])/batsman_stats_df['balls_faced'],2)